In [1]:
import pandas as pd
import time
import numpy as np
import re 
from datetime import datetime as dt
from datetime import timedelta 
import random
from pysolar.solar import *

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import NullFormatter
import matplotlib as mpl
import matplotlib.patches as mpatches
mpl.rc('xtick', color='w')
mpl.rc('ytick', color='w')
%matplotlib inline

# Section I - Crash Data 

## 0:Load Crash Data

Import yearly crash data

In [2]:
df2018 = pd.read_csv('./data/df2018_1min_.csv')
df2017 = pd.read_csv('./data/df2017_1min_.csv')
df2016 = pd.read_csv('./data/df2016_1min_.csv')
df2015 = pd.read_csv('./data/df2015_1min_.csv')
df2014 = pd.read_csv('./data/df2014_1min_.csv')
df2013 = pd.read_csv('./data/df2013_1min_.csv')

In [3]:
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',100000)

Review the number of crash by year (2013 - 2018)

In [4]:
df = pd.concat([df2013, df2014, df2015, df2016, df2017, df2018]).reset_index()

In [5]:
# Number of rows and columns
df.shape

(5915, 217)

Filter unnessary columns

In [6]:
miscols = df.filter(regex ='^Unnamed:').columns
df.drop(columns = miscols, inplace = True)

The maximum of the milepost is 10.8. If 'MP' > 10.8, replace it with '10.8'

In [7]:
df['MP'].replace(10.9, 10.8, inplace = True)

In [8]:
df['MP'] = df['MP'].round(1)

In [9]:
#Convert timestamp into datetime format
df.filter(regex='^pre_[0-9]+min').columns

Index(['pre_1min', 'pre_2min', 'pre_3min', 'pre_4min', 'pre_5min', 'pre_6min',
       'pre_7min', 'pre_8min', 'pre_9min', 'pre_10min', 'pre_11min',
       'pre_12min', 'pre_13min', 'pre_14min', 'pre_15min'],
      dtype='object')

In [10]:
times1 = ['timestamp']
times2 = [t for t in df.filter(regex='^pre_[0-9]+min').columns]
times = times1 + times2

for time in times:
    df[time] = pd.to_datetime(df[time])

Let's drop unnecessary columns

In [11]:
df.drop(columns=['index','DATE', 'TIME','ROUTE', 'RD_SYS', 'CELL_USE', 'tmc_dn3', 
                 'tmc_up3', 'tmc_dn3_miles', 'tmc_up3_miles'], inplace = True)

No. of NA in 'LOC_DIR'

These rows might be partially used in the crash analysis

In [12]:
df[df['LOC_DIR'].isna()].groupby(df['timestamp'].dt.year)['timestamp'].count()

timestamp
2013     52
2014     41
2015     55
2016     72
2017     63
2018    112
Name: timestamp, dtype: int64

No. of NA in 'spd'

INRIX data is missing for the corresponding location and time

In [13]:
# number of NA in 'spd'
df[df['spd'].isna()].groupby(df['timestamp'].dt.year)['timestamp'].count()

timestamp
2013    29
2014     7
2015    15
2016    37
2017    23
2018    14
Name: timestamp, dtype: int64

## 1: LATITUDE & LONGITUDE

This info will be extracted from NJDOT Milepost dataframe

In [14]:
mp = pd.read_csv('./data/NJ3_MP.csv')

In [15]:
mp.head()

,OBJECTID_1,Shape,OBJECTID,SRI,ROUTE,MILEPOST,MUNICIPAL,COUNTY,SHAPE_LENG,LONGITUDE,LATITUDE
0,28142,NaN,27947,3,3,0.0,CLIFTON,PASSAIC,93246.39951,-74.189700,40.871118
1,27385,NaN,27800,3,3,0.1,CLIFTON,PASSAIC,93246.39951,-74.188331,40.870114
2,27913,NaN,27815,3,3,0.2,CLIFTON,PASSAIC,93246.39951,-74.186715,40.869338
3,27595,NaN,27868,3,3,0.3,CLIFTON,PASSAIC,93246.39951,-74.185176,40.868524
4,27465,NaN,27936,3,3,0.4,CLIFTON,PASSAIC,93246.39951,-74.183988,40.867379


In [16]:
#Join tables
df = pd.merge(df, mp, how = 'left', left_on = 'MP', right_on = 'MILEPOST')
df.drop(columns = ['OBJECTID_1', 'Shape', 'OBJECTID', 'SRI', 'ROUTE', 'MILEPOST', 'MUNICIPAL', 'COUNTY', 'SHAPE_LENG'], 
        inplace = True)

## 2:Solar elevation and azimuth

Make 'timestamp' aware time zone:US/Eastern

In [17]:
df['timestamp_tz'] = df['timestamp'].dt.tz_localize(tz='US/Eastern')

In [18]:
df['solar_altitude'] = list(map(lambda x, y, z: get_altitude(x, y, z), df['LATITUDE'], df['LONGITUDE'], df['timestamp_tz'] ))
df['solar_azimuth'] =  list(map(lambda x, y, z: get_azimuth(x, y, z), df['LATITUDE'], df['LONGITUDE'], df['timestamp_tz'] ))

In [19]:
df['timestamp'].dt.dayofweek.value_counts()

1    1011
3    1000
2     990
4     971
0     850
5     608
6     582
Name: timestamp, dtype: int64

## 3:Assign Vertical Profile

Road Grade

In [20]:
g = pd.read_csv('./data/NJ3_grade.csv')

In [21]:
g.index = pd.IntervalIndex.from_arrays(g['MP ST'],g['MP END'],closed='both')
df['grade'] = df['MP'].apply(lambda x: g.iloc[g.index.get_loc(x)]['Grade EB'])
df['grade'] = np.where(df['LOC_DIR']=='W', df['grade']*-1, df['grade'])

In [22]:
df = df.reset_index(drop=True)

In [23]:
df.isna().sum(axis=0)

DAY                 0
KILLED              0
INJURED             0
PED_KILL            0
PED_INJ             0
SEVERITY            0
ALCOHOL             0
HAZMAT              0
CRASH_TYPE          1
VEH_COUNT           0
LOCATION            0
LOC_DIR           405
MP                  0
RD_SURF             5
SURF_COND          10
LIGHT               7
WEATHER             8
RD_DIV             19
RD_WRKZN           81
TIME_ROUND          0
TMC                 0
timestamp           0
tmc_dn1             0
tmc_dn2             0
tmc_up1             0
tmc_up2             0
tmc_miles           0
tmc_dn1_miles       0
tmc_dn2_miles       0
tmc_up1_miles       0
tmc_up2_miles       0
spd               127
dn1_spd           148
dn2_spd           158
up1_spd           144
up2_spd           135
pre_1min            0
pre1_spd          110
pre1_dn1_spd      133
pre1_dn2_spd      144
pre1_up1_spd      129
pre1_up2_spd      121
pre_2min            0
pre2_spd          119
pre2_dn1_spd      143
pre2_dn2_s

It seems many INRIX data points are missing. These will be removed from the dataframe

In [24]:
df.sample(3)

,DAY,KILLED,INJURED,PED_KILL,PED_INJ,SEVERITY,ALCOHOL,HAZMAT,CRASH_TYPE,VEH_COUNT,LOCATION,LOC_DIR,MP,RD_SURF,SURF_COND,LIGHT,WEATHER,RD_DIV,RD_WRKZN,TIME_ROUND,TMC,timestamp,tmc_dn1,tmc_dn2,tmc_up1,tmc_up2,tmc_miles,tmc_dn1_miles,tmc_dn2_miles,tmc_up1_miles,tmc_up2_miles,spd,dn1_spd,dn2_spd,up1_spd,up2_spd,pre_1min,pre1_spd,pre1_dn1_spd,pre1_dn2_spd,pre1_up1_spd,pre1_up2_spd,pre_2min,pre2_spd,pre2_dn1_spd,pre2_dn2_spd,pre2_up1_spd,pre2_up2_spd,pre_3min,pre3_spd,pre3_dn1_spd,pre3_dn2_spd,pre3_up1_spd,pre3_up2_spd,pre_4min,pre4_spd,pre4_dn1_spd,pre4_dn2_spd,pre4_up1_spd,pre4_up2_spd,pre_5min,pre5_spd,pre5_dn1_spd,pre5_dn2_spd,pre5_up1_spd,pre5_up2_spd,pre_6min,pre6_spd,pre6_dn1_spd,pre6_dn2_spd,pre6_up1_spd,pre6_up2_spd,pre_7min,pre7_spd,pre7_dn1_spd,pre7_dn2_spd,pre7_up1_spd,pre7_up2_spd,pre_8min,pre8_spd,pre8_dn1_spd,pre8_dn2_spd,pre8_up1_spd,pre8_up2_spd,pre_9min,pre9_spd,pre9_dn1_spd,pre9_dn2_spd,pre9_up1_spd,pre9_up2_spd,pre_10min,pre10_spd,pre10_dn1_spd,pre10_dn2_spd,pre10_up1_spd,pre10_up2_spd,pre_11min,pre11_spd,pre11_dn1_spd,pre11_dn2_spd,pre11_up1_spd,pre11_up2_spd,pre_12min,pre12_spd,pre12_dn1_spd,pre12_dn2_spd,pre12_up1_spd,pre12_up2_spd,pre_13min,pre13_spd,pre13_dn1_spd,pre13_dn2_spd,pre13_up1_spd,pre13_up2_spd,pre_14min,pre14_spd,pre14_dn1_spd,pre14_dn2_spd,pre14_up1_spd,pre14_up2_spd,pre_15min,pre15_spd,pre15_dn1_spd,pre15_dn2_spd,pre15_up1_spd,pre15_up2_spd,LONGITUDE,LATITUDE,timestamp_tz,solar_altitude,solar_azimuth,grade
5203,SU,0,0,0,0,P,N,N,1.0,2,NJ 3,W,7.5,2.0,2.0,6.0,2.0,1.0,1.0,20:00,120+04735,2018-12-02 20:02:00,120P04735,120+04736,120P04734,120+04734,0.301162,0.297825,0.036315,0.207492,0.522380,66.0,66.0,66.0,66.0,63.0,2018-12-02 20:01:00,66.0,66.0,66.0,66.0,63.0,2018-12-02 20:00:00,66.0,66.0,66.0,66.0,63.0,2018-12-02 19:59:00,66.0,66.0,66.0,66.0,63.0,2018-12-02 19:58:00,66.0,66.0,66.0,66.0,63.0,2018-12-02 19:57:00,66.0,66.0,66.0,66.0,63.0,2018-12-02 19:56:00,65.0,65.0,63.0,64.0,63.0,2018-12-02 19:55:00,63.0,61.0,63.0,64.0,63.0,2018-12-02 19:54:00,64.0,61.0,63.0,63.0,63.0,2018-12-02 19:53:00,64.0,61.0,63.0,63.0,60.0,2018-12-02 19:52:00,64.0,61.0,63.0,63.0,60.0,2018-12-02 19:51:00,64.0,63.0,63.0,63.0,60.0,2018-12-02 19:50:00,64.0,61.0,63.0,63.0,60.0,2018-12-02 19:49:00,54.0,56.0,57.0,56.0,56.0,2018-12-02 19:48:00,54.0,56.0,57.0,56.0,53.0,2018-12-02 19:47:00,54.0,56.0,59.0,56.0,53.0,-74.081249,40.809295,2018-12-02 20:02:00-05:00,-39.649675,274.075830,0.3
1857,W,0,0,0,0,P,N,N,99.0,2,NJ 3,E,0.0,2.0,1.0,1.0,1.0,1.0,1.0,6:50,120-07448,2015-11-25 06:49:00,120N07448,120-07447,120+06052,120P06052,0.401996,0.332592,0.394906,0.316230,0.243952,56.0,60.0,61.0,57.0,56.0,2015-11-25 06:48:00,55.0,58.0,59.0,56.0,56.0,2015-11-25 06:47:00,55.0,58.0,59.0,56.0,56.0,2015-11-25 06:46:00,55.0,58.0,59.0,56.0,56.0,2015-11-25 06:45:00,55.0,58.0,59.0,51.0,55.0,2015-11-25 06:44:00,55.0,58.0,59.0,51.0,56.0,2015-11-25 06:43:00,55.0,58.0,59.0,52.0,55.0,2015-11-25 06:42:00,55.0,58.0,59.0,52.0,54.0,2015-11-25 06:41:00,55.0,58.0,59.0,52.0,54.0,2015-11-25 06:40:00,55.0,58.0,59.0,52.0,54.0,2015-11-25 06:39:00,55.0,58.0,59.0,52.0,54.0,2015-11-25 06:38:00,49.0,55.0,59.0,52.0,53.0,2015-11-25 06:37:00,52.0,56.0,59.0,50.0,52.0,2015-11-25 06:36:00,52.0,56.0,59.0,50.0,52.0,2015-11-25 06:35:00,53.0,54.0,57.0,51.0,52.0,2015-11-25 06:34:00,62.0,58.0,59.0,53.0,60.0,-74.189700,40.871118,2015-11-25 06:49:00-05:00,-1.872445,116.110523,-2.7
2405,TH,0,0,0,0,P,N,N,11.0,1,NJ 3,NaN,9.8,2.0,3.0,1.0,3.0,2.0,1.0,11:05,120+04731,2015-03-05 11:06:00,120P04731,120+04732,120+04730,120P04729,0.140287,0.011776,0.628204,0.297775,0.429524,15.0,15.0,15.0,15.0,17.0,2015-03-05 11:05:00,16.0,16.0,17.0,16.0,17.0,2015-03-05 11:04:00,16.0,16.0,16.0,16.0,17.0,2015-03-05 11:03:00,15.0,15.0,16.0,15.0,17.0,2015-03-05 11:02:00,15.0,15.0,16.0,15.0,17.0,2015-03-05 11:01:00,16.0,16.0,23.0,16.0,16.0,2015-03-05 11:00:00,22.0,30.0,33.0,16.0,22.0,2015-03-05 10:59:00,22.0,30.0,33.0,16.0,22.0,2015-03-05 10:58:00,35.0,35.0,35.0,29.0,28.0,2015-03-05 10:57:00,35.0,35.0,35.0,29.0,28.0,2015-03-

## 4: Speed columns postprocessing

In [25]:
#speed difference between crash link and downstream or upstream TMCs
df['dn1_spd_diff'] = df['spd'] - df['dn1_spd']
df['dn2_spd_diff'] = df['spd'] - df['dn2_spd']

df['up1_spd_diff'] = df['spd'] - df['up1_spd']
df['up2_spd_diff'] = df['spd'] - df['up2_spd']

df['up1dn1_spd_diff'] = df['up1_spd'] - df['dn1_spd']
df['up1dn2_spd_diff'] = df['up1_spd'] - df['dn2_spd']

df['up2dn1_spd_diff'] = df['up1_spd'] - df['dn1_spd']
df['up2dn2_spd_diff'] = df['up1_spd'] - df['dn2_spd']

### Mean, Std, and CV of speed for 5, 10, and 15 mins 

In [26]:
#Mean Speed for the previous 15 minutes

def spd_mean_std_cv(df):
    
    crash_cols = ['pre1_spd','pre2_spd','pre3_spd','pre4_spd','pre5_spd',
                  'pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd', 
                  'pre11_spd','pre12_spd','pre13_spd','pre14_spd','pre15_spd']
    
    dn1_cols   = ['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd','pre4_dn1_spd','pre5_dn1_spd',
                  'pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd','pre9_dn1_spd','pre10_dn1_spd',
                  'pre11_dn1_spd','pre12_dn1_spd','pre13_dn1_spd','pre14_dn1_spd','pre15_dn1_spd']
    
    dn2_cols   = ['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd','pre4_dn2_spd','pre5_dn2_spd',
                  'pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd','pre9_dn2_spd','pre10_dn2_spd',
                  'pre11_dn2_spd','pre12_dn2_spd','pre13_dn2_spd','pre14_dn2_spd','pre15_dn2_spd'] 
    
    up1_cols   = ['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd','pre4_up1_spd','pre5_up1_spd',
                  'pre6_up1_spd','pre7_up1_spd','pre8_up1_spd','pre9_up1_spd','pre10_up1_spd',
                  'pre11_up1_spd','pre12_up1_spd','pre13_up1_spd','pre14_up1_spd','pre15_up1_spd']
    
    up2_cols   = ['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd','pre4_up2_spd','pre5_up2_spd',
                  'pre6_up2_spd','pre7_up2_spd','pre8_up2_spd','pre9_up2_spd','pre10_up2_spd',
                  'pre11_up2_spd','pre12_up2_spd','pre13_up2_spd','pre14_up2_spd','pre15_up2_spd']
    
    minutes =[5, 10, 15]
    #mean, stdev, and cv of speed for 5, 10, 15 minutes 
    for minute in minutes:
        if minute == 5:
            i = 5
        elif minute == 10:
            i = 10
        else: minute = 15
        df['avg_spd_%dmin' % minute]     = df[crash_cols[:i]].mean(skipna=True, axis=1)
        df['avg_spd_dn1_%dmin' % minute] = df[dn1_cols[:i]].mean(skipna=True, axis=1)
        df['avg_spd_dn2_%dmin' % minute] = df[dn2_cols[:i]].mean(skipna=True, axis=1)
        df['avg_spd_up1_%dmin' % minute] = df[up1_cols[:i]].mean(skipna=True, axis=1)
        df['avg_spd_up2_%dmin' % minute] = df[up2_cols[:i]].mean(skipna=True, axis=1)
        df['std_%dmin' % minute]         = df[crash_cols[:i]].std(skipna=True, axis=1)
        df['std_dn1_%dmin' % minute]     = df[dn1_cols[:i]].std(skipna=True, axis=1)
        df['std_dn2_%dmin' % minute]     = df[dn2_cols[:i]].std(skipna=True, axis=1)
        df['std_up1_%dmin' % minute]     = df[up1_cols[:i]].std(skipna=True, axis=1)
        df['std_up2_%dmin' % minute]     = df[up2_cols[:i]].std(skipna=True, axis=1)
        df['cv_%dmin' % minute]          = df['std_%dmin' % minute] / df['avg_spd_%dmin' % minute]
        df['cv_dn1_%dmin' % minute]      = df['std_dn1_%dmin' % minute] / df['avg_spd_dn1_%dmin' % minute]
        df['cv_dn2_%dmin' % minute]      = df['std_dn2_%dmin' % minute] / df['avg_spd_dn2_%dmin' % minute]
        df['cv_up1_%dmin' % minute]      = df['std_up1_%dmin' % minute] / df['avg_spd_up1_%dmin' % minute]
        df['cv_up2_%dmin' % minute]      = df['std_up2_%dmin' % minute] / df['avg_spd_up2_%dmin' % minute]
        
    return df

In [27]:
df = spd_mean_std_cv(df)

### Speed difference between upstream and downstrem link

In [28]:
df['avg_spd_diff_up1dn1_15min'] = df['avg_spd_up1_15min'].sub(df['avg_spd_dn1_15min'], axis = 0) 
df['avg_spd_diff_up2dn2_15min'] = df['avg_spd_up2_15min'].sub(df['avg_spd_dn2_15min'], axis = 0) 

df['avg_spd_diff_up1dn1_10min'] = df['avg_spd_up1_10min'].sub(df['avg_spd_dn1_10min'], axis = 0) 
df['avg_spd_diff_up2dn2_10min'] = df['avg_spd_up2_10min'].sub(df['avg_spd_dn2_10min'], axis = 0) 

df['avg_spd_diff_up1dn1_5min'] = df['avg_spd_up1_5min'].sub(df['avg_spd_dn1_5min'], axis = 0) 
df['avg_spd_diff_up2dn2_5min'] = df['avg_spd_up2_5min'].sub(df['avg_spd_dn2_5min'], axis = 0) 

In [29]:
df['spd_up1dn1_diff_TMC_len_15min'] = df['avg_spd_diff_up1dn1_15min']/df['tmc_miles'] 
df['spd_up2dn2_diff_TMC_len_15min'] = df['avg_spd_diff_up2dn2_15min']/df['tmc_miles'] 

df['spd_up1dn1_diff_TMC_len_10min'] = df['avg_spd_diff_up1dn1_10min']/df['tmc_miles'] 
df['spd_up2dn2_diff_TMC_len_10min'] = df['avg_spd_diff_up2dn2_10min']/df['tmc_miles'] 

df['spd_up1dn1_diff_TMC_len_5min'] = df['avg_spd_diff_up1dn1_5min']/df['tmc_miles'] 
df['spd_up2dn2_diff_TMC_len_5min'] = df['avg_spd_diff_up2dn2_5min']/df['tmc_miles'] 

## 5:Join Weather Data

In [30]:
wdf = pd.read_csv('./data/weather_hist.csv')

In [31]:
wdf.head()

,Date,High,Low,Precip.,Snow,Snow d.
0,1/1/2013,41.0,27.0,T,T,0
1,1/2/2013,35.1,24.1,0,0,0
2,1/3/2013,34.0,26.1,0,0,0
3,1/4/2013,39.9,28.0,0,0,0
4,1/5/2013,44.1,30.9,0,0,0


In [32]:
wdf['Date'] = pd.to_datetime(wdf['Date'])

In [33]:
wdf['Precip.'].replace('T', 0.0, inplace = True)
wdf['Snow'].replace('T', 0.0, inplace = True)
wdf['Snow'].replace('-', 0.0, inplace = True)
wdf['Snow d.'].replace('-', 0.0, inplace = True)
wdf['Snow d.'].replace('T', 0.0, inplace = True)
wdf.fillna(0, inplace = True)

In [34]:
wdf['Date'] = wdf['Date'].astype(str)

In [35]:
df['DATE'] = df['timestamp'].dt.date
df['DATE'] = df['DATE'].astype(str)

In [36]:
df = pd.merge(df, wdf, how = 'left', left_on = 'DATE', right_on = 'Date' )

In [37]:
df.drop(columns = ['DATE'], inplace = True)

## 6:AADT

In [38]:
aadt = pd.read_csv('./data/NJCMS_Rt3.csv')

In [39]:
aadt.sample(3)

,LINKID,ROUTE,SRI_CMS,BEGINMP,ENDMP,SOURCEID,ANODE,BNODE,DIST,FTYPE,ATYPE,LANES,FCLASS,CDEV,ZONE,NETVOL24,NETVOLAM,NETVOLPM,CAPLINK,ZVTLINK,ZVTCDEV,ZVTOTAL,VOL00,VOL01,VOL02,VOL03,VOL04,VOL05,VOL06,VOL07,VOL08,VOL09,VOL10,VOL11,VOL12,VOL13,VOL14,VOL15,VOL16,VOL17,VOL18,VOL19,VOL20,VOL21,VOL22,VOL23,VOL24,TRK00,TRK01,TRK02,TRK03,TRK04,TRK05,TRK06,TRK07,TRK08,TRK09,TRK10,TRK11,TRK12,TRK13,TRK14,TRK15,TRK16,TRK17,TRK18,TRK19,TRK20,TRK21,TRK22,TRK23,TRK24,SPD00,SPD01,SPD02,SPD03,SPD04,SPD05,SPD06,SPD07,SPD08,SPD09,SPD10,SPD11,SPD12,SPD13,SPD14,SPD15,SPD16,SPD17,SPD18,SPD19,SPD20,SPD21,SPD22,SPD23,SPD24,LVC00,LVC01,LVC02,LVC03,LVC04,LVC05,LVC06,LVC07,LVC08,LVC09,LVC10,LVC11,LVC12,LVC13,LVC14,LVC15,LVC16,LVC17,LVC18,LVC19,LVC20,LVC21,LVC22,LVC23,LVC24,AGROUP,FGROUP,RGROUP,NOTES,CAR00,CAR01,CAR02,CAR03,CAR04,CAR05,CAR06,CAR07,CAR08,CAR09,CAR10,CAR11,CAR12,CAR13,CAR14,CAR15,CAR16,CAR17,CAR18,CAR19,CAR20,CAR21,CAR22,CAR23,WEEKEND_FACTOR
3,1000247,NJ-3,00000003__,1.26,1.53,1218,27470,21120,0.27,2,3,3,12,0,31,71781,5412,4740,4904,17.7,0,17.7,681,344,303,280,526,1456,4294,5412,5004,3633,3114,3214,3437,3445,3694,4740,4740,4740,4740,4367,3033,3016,2189,1383,71785,121,61,54,24,42,121,763,962,890,646,631,651,680,714,717,726,741,568,843,776,539,536,389,246,12441,54.92,54.92,54.92,54.92,54.92,54.92,39.51,8.86,19.60,48.12,51.70,51.16,49.85,49.34,47.41,32.73,32.62,33.99,32.29,38.42,52.54,52.54,54.61,54.92,30.86,0.139,0.070,0.062,0.056,0.105,0.289,0.878,1.107,1.024,0.743,0.645,0.666,0.706,0.714,0.758,0.964,0.965,0.949,0.970,0.893,0.620,0.617,0.448,0.283,0,31,2,0,NaN,560,283,249,256,484,1335,3531,4450,4114,2987,2483,2563,2757,2731,2977,4014,3999,4172,3897,3591,2494,2480,1800,1137,1.00624
17,1000275,NJ-3,00000003__,9.05,9.12,1246,27730,21460,0.02,2,2,4,12,0,17,71288,6508,6774,6619,1.3,0,1.3,659,324,283,261,239,749,4814,6508,5407,2978,2668,3176,3413,3421,3134,3641,6641,6774,4331,2849,2479,2995,2175,1373,71292,65,36,30,108,80,258,397,590,517,382,307,512,486,234,352,498,636,520,462,304,265,311,226,143,7719,55.38,55.38,55.38,55.38,55.38,55.38,48.00,32.73,45.00,55.38,55.38,55.38,55.38,55.38,55.38,55.38,30.00,8.09,51.43,55.38,55.38,55.38,55.38,55.38,32.01,0.098,0.049,0.042,0.043,0.038,0.120,0.717,0.971,0.807,0.447,0.400,0.485,0.516,0.508,0.470,0.550,0.991,1.009,0.647,0.426,0.370,0.447,0.325,0.205,0,17,2,0,NaN,594,288,253,153,159,491,4417,5918,4890,2596,2361,2664,2927,3187,2782,3143,6005,6254,3869,2545,2214,2684,1949,1230,1.00462
9,1000259,NJ-3,00000003__,5.01,5.67,1230,27510,27550,0.66,2,3,3,12,0,3,70018,5295,5174,4871,43.2,0,43.2,1651,1325,1310,1292,1358,1499,2733,5090,5295,3603,3081,3175,3426,3539,3354,3401,4884,5174,4017,2758,2274,2170,1958,1647,70014,226,182,192,710,595,823,312,629,701,627,494,729,724,305,506,733,646,518,607,416,343,328,296,264,11906,55.00,55.00,55.00,55.00,55.00,54.87,53.76,25.33,18.62,48.39,52.22,51.32,49.60,49.81,50.66,49.71,31.10,23.32,43.92,53.63,54.50,54.62,54.87,54.87,38.29,0.333,0.267,0.264,0.291,0.298,0.338,0.547,1.019,1.066,0.737,0.628,0.662,0.711,0.704,0.682,0.707,0.983,1.030,0.817,0.561,0.462,0.441,0.398,0.336,0,3,2,0,NaN,1425,1143,1118,582,763,676,2421,4461,4594,2976,2587,2446,2702,3234,2848,2668,4238,4656,3410,2342,1931,1842,1662,1383,1.06710


In [40]:
aadt.drop(columns = 
           ['LINKID',
            'ROUTE',
            'SOURCEID',
            'ANODE',
            'BNODE',
            'FTYPE',
            'ATYPE',
            'FCLASS',
            'CDEV',
            'ZONE',
            'NETVOL24',
            'NETVOLAM',
            'NETVOLPM',
            'ZVTLINK',
            'ZVTCDEV',
            'ZVTOTAL',
            'SPD00',
            'SPD01',
            'SPD02',
            'SPD03',
            'SPD04',
            'SPD05',
            'SPD06',
            'SPD07',
            'SPD08',
            'SPD09',
            'SPD10',
            'SPD11',
            'SPD12',
            'SPD13',
            'SPD14',
            'SPD15',
            'SPD16',
            'SPD17',
            'SPD18',
            'SPD19',
            'SPD20',
            'SPD21',
            'SPD22',
            'SPD23',
            'SPD24',
            'LVC00',
            'LVC01',
            'LVC02',
            'LVC03',
            'LVC04',
            'LVC05',
            'LVC06',
            'LVC07',
            'LVC08',
            'LVC09',
            'LVC10',
            'LVC11',
            'LVC12',
            'LVC13',
            'LVC14',
            'LVC15',
            'LVC16',
            'LVC17',
            'LVC18',
            'LVC19',
            'LVC20',
            'LVC21',
            'LVC22',
            'LVC23',
            'LVC24',
            'AGROUP',
            'FGROUP',
            'RGROUP',
            'NOTES',
            'CAR00',
            'CAR01',
            'CAR02',
            'CAR03',
            'CAR04',
            'CAR05',
            'CAR06',
            'CAR07',
            'CAR08',
            'CAR09',
            'CAR10',
            'CAR11',
            'CAR12',
            'CAR13',
            'CAR14',
            'CAR15',
            'CAR16',
            'CAR17',
            'CAR18',
            'CAR19',
            'CAR20',
            'CAR21',
            'CAR22',
            'CAR23'], inplace = True)

In [41]:
aadt_E = aadt[aadt['SRI_CMS'] == '00000003__']
aadt_W = aadt[aadt['SRI_CMS'] == '00000003_W']

In [42]:
df_E = df[df['LOC_DIR']=='E']
df_W = df[df['LOC_DIR']=='W']

In [43]:
aadt_E.index = pd.IntervalIndex.from_arrays(aadt_E['BEGINMP'], aadt_E['ENDMP'],closed='both')
aadt_W.index = pd.IntervalIndex.from_arrays(aadt_W['ENDMP'], aadt_W['BEGINMP'],closed='both')

In [44]:
df_E['begin_MP'] = df_E['MP'].apply(lambda x: aadt_E.iloc[aadt_E.index.get_loc(x)]['BEGINMP'])
df_W['begin_MP'] = df_W['MP'].apply(lambda x: aadt_W.iloc[aadt_W.index.get_loc(x)]['BEGINMP'])

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
aadt['DIR'] = aadt['SRI_CMS'].apply(lambda d: 'E' if d == '00000003__' else 'W')

In [46]:
aadt['BEGINMP'] = aadt['BEGINMP'].astype(str)

In [47]:
df_E['begin_MP'] = df_E['begin_MP'].astype(str)
df_W['begin_MP'] = df_W['begin_MP'].astype(str)

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
df1 = pd.merge(df_E, aadt, how = 'left', left_on = ['LOC_DIR', 'begin_MP'], right_on = ['DIR', 'BEGINMP'])
df2 = pd.merge(df_W, aadt, how = 'left', left_on = ['LOC_DIR', 'begin_MP'], right_on = ['DIR', 'BEGINMP'])

In [49]:
print("Total number of rows:{} and columns:{} in the eastbound dataset".format(df1.shape[0], df1.shape[1]))

Total number of rows:2361 and columns:262 in the eastbound dataset


In [50]:
print("Total number of rows:{} and columns:{} in the westbound dataset".format(df2.shape[0], df2.shape[1]))

Total number of rows:3371 and columns:262 in the westbound dataset


In [51]:
df = pd.concat([df1, df2]).reset_index(drop=True)

In [52]:
df['HOUR'] = df['timestamp'].dt.hour
df['HOUR'] = df['HOUR'].map("{:02}".format)

In [53]:
df['vol_AADT_ref'] = 'VOL' + df['HOUR']
df['trk_AADT_ref'] = 'TRK' + df['HOUR']

In [54]:
df['VOL_AADT'] = df.lookup(df.index, df['vol_AADT_ref']) 
df['TRK_AADT'] = df.lookup(df.index, df['trk_AADT_ref']) 

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  """Entry point for launching an IPython kernel.
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  


In [55]:
df['VOL'] = df['VOL_AADT']
df['VOL'] =np.where(df['DAY']=='SA', df['VOL_AADT']/df['WEEKEND_FACTOR'], df['VOL'])
df['VOL'] =np.where(df['DAY']=='S', df['VOL_AADT']/df['WEEKEND_FACTOR'], df['VOL'])

In [56]:
df['TRK'] = df['TRK_AADT']
df['TRK'] =np.where(df['DAY']=='SA', df['TRK_AADT']/df['WEEKEND_FACTOR'], df['TRK'])
df['TRK'] =np.where(df['DAY']=='S', df['TRK_AADT']/df['WEEKEND_FACTOR'], df['TRK'])

In [57]:
df['VOL'] = df['VOL'].round()
df['TRK'] = df['TRK'].round()

In [58]:
df.drop(columns = ['begin_MP',
'SRI_CMS',
'BEGINMP',
'ENDMP',
'DIST',
'VOL00',
'VOL01',
'VOL02',
'VOL03',
'VOL04',
'VOL05',
'VOL06',
'VOL07',
'VOL08',
'VOL09',
'VOL10',
'VOL11',
'VOL12',
'VOL13',
'VOL14',
'VOL15',
'VOL16',
'VOL17',
'VOL18',
'VOL19',
'VOL20',
'VOL21',
'VOL22',
'VOL23',
'VOL24',
'TRK00',
'TRK01',
'TRK02',
'TRK03',
'TRK04',
'TRK05',
'TRK06',
'TRK07',
'TRK08',
'TRK09',
'TRK10',
'TRK11',
'TRK12',
'TRK13',
'TRK14',
'TRK15',
'TRK16',
'TRK17',
'TRK18',
'TRK19',
'TRK20',
'TRK21',
'TRK22',
'TRK23',
'TRK24',
'WEEKEND_FACTOR',
'DIR',
'HOUR',
'vol_AADT_ref',
'trk_AADT_ref',
'VOL_AADT',
'TRK_AADT'
], inplace = True)

In [59]:
df['Precip.'] = df['Precip.'].astype(float)
df['Snow'] = df['Snow'].astype(float)
df['Snow d.'] = df['Snow d.'].astype(float)

In [60]:
df.to_csv('df_all_processed.csv')

In [61]:
df['SEVERITY'].value_counts()

P    4384
I    1330
F      18
Name: SEVERITY, dtype: int64

# Section II - Normal Data Sampling


## 0:Load Non-crash Data

In [62]:
ndf2018 = pd.read_csv('./data/n2018_.csv')
ndf2017 = pd.read_csv('./data/n2017_.csv')
ndf2016 = pd.read_csv('./data/n2016_.csv')
ndf2015 = pd.read_csv('./data/n2015_.csv')
ndf2014 = pd.read_csv('./data/n2014_.csv')
ndf2013 = pd.read_csv('./data/n2013_.csv')

In [63]:
ndfs = [ndf2018, ndf2017, ndf2016, ndf2015, ndf2014, ndf2013]

for i in ndfs:
    miscols = i.filter(regex ='^Unnamed:').columns
    i.drop(columns = miscols, inplace = True)

In [64]:
# remove 3rd upstream and downstream links

ndf2018.drop(columns = ['tmc_dn3', 'tmc_up3', 'tmc_dn3_miles', 'tmc_up3_miles'] , inplace = True)
ndf2017.drop(columns = ['tmc_dn3', 'tmc_up3', 'tmc_dn3_miles', 'tmc_up3_miles'] , inplace = True)
ndf2016.drop(columns = ['tmc_dn3', 'tmc_up3', 'tmc_dn3_miles', 'tmc_up3_miles'] , inplace = True)
ndf2015.drop(columns = ['tmc_dn3', 'tmc_up3', 'tmc_dn3_miles', 'tmc_up3_miles'] , inplace = True)
ndf2013.drop(columns = ['tmc_dn3', 'tmc_up3'] , inplace = True)

In [65]:
#Non-crash data size is too large. Reduce the sample size

ndf2018 = ndf2018.sample(n=5800)
ndf2017 = ndf2017.sample(n=4850)
ndf2016 = ndf2016.sample(n=3500)
ndf2015 = ndf2015.sample(n=3000)
ndf2014 = ndf2014.sample(n=3200)
ndf2013 = ndf2013.sample(n=3700)

In [66]:
ndfs = [ndf2018, ndf2017, ndf2016, ndf2015, ndf2014, ndf2013]

for i in ndfs:
    print(i.shape)

(5800, 106)
(4850, 106)
(3500, 106)
(3000, 106)
(3200, 106)
(3700, 106)


In [67]:
ndf = pd.concat([ndf2018, ndf2017, ndf2016, ndf2015, ndf2014, ndf2013]).reset_index()
ndf.to_csv('ndf_.csv')
ndf_TMC = pd.read_csv('./data/ndf_TMC.csv')

In [68]:
ndf.head()

,index,timestamp,TMC,tmc_dn1,tmc_dn2,tmc_up1,tmc_up2,tmc_miles,tmc_dn1_miles,tmc_dn2_miles,tmc_up1_miles,tmc_up2_miles,spd,dn1_spd,dn2_spd,up1_spd,up2_spd,pre_1min,pre1_spd,pre1_dn1_spd,pre1_dn2_spd,pre1_up1_spd,pre1_up2_spd,pre_2min,pre2_spd,pre2_dn1_spd,pre2_dn2_spd,pre2_up1_spd,pre2_up2_spd,pre_3min,pre3_spd,pre3_dn1_spd,pre3_dn2_spd,pre3_up1_spd,pre3_up2_spd,pre_4min,pre4_spd,pre4_dn1_spd,pre4_dn2_spd,pre4_up1_spd,pre4_up2_spd,pre_5min,pre5_spd,pre5_dn1_spd,pre5_dn2_spd,pre5_up1_spd,pre5_up2_spd,pre_6min,pre6_spd,pre6_dn1_spd,pre6_dn2_spd,pre6_up1_spd,pre6_up2_spd,pre_7min,pre7_spd,pre7_dn1_spd,pre7_dn2_spd,pre7_up1_spd,pre7_up2_spd,pre_8min,pre8_spd,pre8_dn1_spd,pre8_dn2_spd,pre8_up1_spd,pre8_up2_spd,pre_9min,pre9_spd,pre9_dn1_spd,pre9_dn2_spd,pre9_up1_spd,pre9_up2_spd,pre_10min,pre10_spd,pre10_dn1_spd,pre10_dn2_spd,pre10_up1_spd,pre10_up2_spd,pre_11min,pre11_spd,pre11_dn1_spd,pre11_dn2_spd,pre11_up1_spd,pre11_up2_spd,pre_12min,pre12_spd,pre12_dn1_spd,pre12_dn2_spd,pre12_up1_spd,pre12_up2_spd,pre_13min,pre13_spd,pre13_dn1_spd,pre13_dn2_spd,pre13_up1_spd,pre13_up2_spd,pre_14min,pre14_spd,pre14_dn1_spd,pre14_dn2_spd,pre14_up1_spd,pre14_up2_spd,pre_15min,pre15_spd,pre15_dn1_spd,pre15_dn2_spd,pre15_up1_spd,pre15_up2_spd
0,7469,2018-05-26 13:22:00,120-04731,120N04731,120-04730,120N04732,120-04732,0.542457,0.079098,0.137006,0.025403,0.333586,62.0,65.0,63.0,58.0,61.0,2018-05-26 13:21:00,63.0,67.0,60.0,53.0,60.0,2018-05-26 13:20:00,63.0,67.0,60.0,53.0,60.0,2018-05-26 13:19:00,63.0,67.0,60.0,54.0,53.0,2018-05-26 13:18:00,67.0,67.0,60.0,63.0,57.0,2018-05-26 13:17:00,67.0,67.0,60.0,63.0,57.0,2018-05-26 13:16:00,68.0,67.0,60.0,62.0,61.0,2018-05-26 13:15:00,64.0,66.0,60.0,62.0,61.0,2018-05-26 13:14:00,65.0,66.0,60.0,62.0,61.0,2018-05-26 13:13:00,65.0,66.0,60.0,62.0,61.0,2018-05-26 13:12:00,56.0,59.0,58.0,53.0,54.0,2018-05-26 13:11:00,56.0,59.0,58.0,53.0,54.0,2018-05-26 13:10:00,56.0,59.0,58.0,53.0,54.0,2018-05-26 13:09:00,56.0,59.0,58.0,53.0,54.0,2018-05-26 13:08:00,55.0,58.0,57.0,52.0,54.0,2018-05-26 13:07:00,55.0,58.0,57.0,52.0,54.0
1,4306,2018-09-16 09:53:00,120-07443,120N07443,120-07442,120N07444,120-07444,0.674157,0.068105,0.346611,0.150764,0.832980,64.0,64.0,62.0,70.0,67.0,2018-09-16 09:52:00,64.0,64.0,60.0,70.0,68.0,2018-09-16 09:51:00,65.0,65.0,60.0,70.0,67.0,2018-09-16 09:50:00,66.0,65.0,62.0,69.0,68.0,2018-09-16 09:49:00,66.0,65.0,62.0,70.0,67.0,2018-09-16 09:48:00,68.0,67.0,63.0,72.0,68.0,2018-09-16 09:47:00,68.0,67.0,63.0,72.0,68.0,2018-09-16 09:46:00,68.0,67.0,63.0,72.0,68.0,2018-09-16 09:45:00,69.0,69.0,68.0,72.0,68.0,2018-09-16 09:44:00,71.0,69.0,70.0,73.0,70.0,2018-09-16 09:43:00,70.0,69.0,70.0,72.0,70.0,2018-09-16 09:42:00,70.0,70.0,71.0,72.0,70.0,2018-09-16 09:41:00,68.0,71.0,71.0,67.0,68.0,2018-09-16 09:40:00,68.0,71.0,71.0,67.0,68.0,2018-09-16 09:39:00,68.0,71.0,71.0,67.0,68.0,2018-09-16 09:38:00,65.0,68.0,70.0,68.0,71.0
2,7027,2018-02-02 14:40:00,120P04737,120+04738,120P04738,120+04737,120P04736,0.482554,0.280115,0.381707,0.644361,0.062811,62.0,58.0,60.0,56.0,59.0,2018-02-02 14:39:00,62.0,58.0,60.0,56.0,59.0,2018-02-02 14:38:00,60.0,57.0,56.0,56.0,60.0,2018-02-02 14:37:00,60.0,57.0,56.0,56.0,60.0,2018-02-02 14:36:00,60.0,57.0,54.0,56.0,60.0,2018-02-02 14:35:00,60.0,56.0,54.0,57.0,58.0,2018-02-02 14:34:00,60.0,56.0,54.0,57.0,58.0,2018-02-02 14:33:00,55.0,53.0,53.0,59.0,55.0,2018-02-02 14:32:00,54.0,51.0,51.0,56.0,55.0,2018-02-02 14:31:00,54.0,51.0,51.0,56.0,55.0,2018-02-02 14:30:00,54.0,50.0,64.0,56.0,55.0,2018-02-02 14:29:00,54.0,52.0,54.0,57.0,55.0,2018-02-02 14:28:00,54.0,52.0,54.0,57.0,55.0,2018-02-02 14:27:00,57.0,53.0,54.0,58.0,56.0,2018-02-02 14:26:00,56.0,54.0,54.0,57.0,55.0,2018-02-02 14:25:00,56.0,54.0,54.0,57.0,55.0
3,6689,2018-03-26 06:40:00,120+04739,120P04739,120+04740,120P04738,120+04738,0.221790,0.267140,0.335239,0.381707,0.280115,61.0,64.0,60.0,61.0,54.0,2018-03-26 06:39:00,61.0,64.0,60.0,61.0,54.0,2018-03-26 06:38:00,58.0,66.0,62.0,59.0,54.0,2018-03-26 06:37:00,58.0,66.0,62.0,67.0,69.0,2018-03-26 

In [69]:
ndf = pd.merge(ndf, ndf_TMC, how='left', left_on = 'TMC', right_on = 'tmc')

In [70]:
from random import *

ndf['MP'] = uniform(ndf['ST_MP'], ndf['END_MP'])
ndf['MP'] = ndf['MP'].round(1)

In [71]:
ndf.drop(columns=['tmc', 'ST_MP', 'END_MP'], inplace = True)

## 1:Solar Elevation & Azimuth

In [72]:
ndf = pd.merge(ndf, mp, how = 'left', left_on = 'MP', right_on = 'MILEPOST')
ndf.drop(columns = ['OBJECTID_1', 'Shape', 'OBJECTID', 'SRI', 'ROUTE', 'MILEPOST', 'MUNICIPAL', 'COUNTY', 'SHAPE_LENG'], 
        inplace = True)

In [73]:
time_index = pd.DatetimeIndex(ndf['timestamp'])

ndf['hour'] = time_index.hour
ndf['weekday'] = time_index.weekday
ndf['month'] = time_index.month

In [76]:
ndf['timestamp'] = pd.to_datetime(ndf['timestamp'])

# Data for the following timestamp is not avilable. 
ndf = ndf[ndf['timestamp'] != '2017-03-12 02:39:00']
ndf = ndf[ndf['timestamp'] != '2017-03-12 02:27:00']
ndf = ndf[ndf['timestamp'] != '2018-03-11 02:00:00']
ndf = ndf[ndf['timestamp'] != '2014-03-09 02:22:00']
ndf = ndf[ndf['timestamp'] != '2013-03-10 02:58:00']
ndf = ndf[ndf['timestamp'] != '2013-03-10 02:04:00']
ndf = ndf[ndf['timestamp'] != '2013-03-10 02:20:00']


In [77]:
# Set timezone
ndf['timestamp_tz'] = ndf['timestamp'].dt.tz_localize(tz='US/Eastern', ambiguous=True)

In [78]:
# Solar azimuth and elevation
ndf['solar_altitude'] = list(map(lambda x, y, z: get_altitude(x, y, z), ndf['LATITUDE_x'], ndf['LONGITUDE_x'], ndf['timestamp_tz'] ))
ndf['solar_azimuth'] =  list(map(lambda x, y, z: get_azimuth(x, y, z), ndf['LATITUDE_x'], ndf['LONGITUDE_x'], ndf['timestamp_tz'] ))

In [79]:
ndf['timestamp'].dt.year.value_counts()

2018    5799
2017    4849
2013    3697
2016    3500
2014    3199
2015    3000
Name: timestamp, dtype: int64

## 2:Speed columns postprocessing

In [80]:
#speed difference between crash link and downstream or upstream TMCs
ndf['dn1_spd_diff'] = ndf['spd'] - ndf['dn1_spd']
ndf['dn2_spd_diff'] = ndf['spd'] - ndf['dn2_spd']

ndf['up1_spd_diff'] = ndf['spd'] - ndf['up1_spd']
ndf['up2_spd_diff'] = ndf['spd'] - ndf['up2_spd']

ndf['up1dn1_spd_diff'] = ndf['up1_spd'] - ndf['dn1_spd']
ndf['up1dn2_spd_diff'] = ndf['up1_spd'] - ndf['dn2_spd']

ndf['up2dn1_spd_diff'] = ndf['up1_spd'] - ndf['dn1_spd']
ndf['up2dn2_spd_diff'] = ndf['up1_spd'] - ndf['dn2_spd']

In [81]:
ndf = spd_mean_std_cv(ndf)

## 3:Join Weather Data

In [82]:
ndf['DATE'] = ndf['timestamp'].dt.date
ndf['DATE'] = ndf['DATE'].astype(str)

In [83]:
ndf = pd.merge(ndf, wdf, how = 'left', left_on = 'DATE', right_on = 'Date' )

In [84]:
ndf.drop(columns = ['DATE'], inplace = True)

## 4:AADT

In [85]:
aadt = pd.read_csv('./data/NJCMS_Rt3.csv')

In [86]:
aadt.drop(columns = ['LINKID',
'ROUTE',
'SOURCEID',
'ANODE',
'BNODE',
'FTYPE',
'ATYPE',
'FCLASS',
'CDEV',
'ZONE',
'NETVOL24',
'NETVOLAM',
'NETVOLPM',
'ZVTLINK',
'ZVTCDEV',
'ZVTOTAL',
'SPD00',
'SPD01',
'SPD02',
'SPD03',
'SPD04',
'SPD05',
'SPD06',
'SPD07',
'SPD08',
'SPD09',
'SPD10',
'SPD11',
'SPD12',
'SPD13',
'SPD14',
'SPD15',
'SPD16',
'SPD17',
'SPD18',
'SPD19',
'SPD20',
'SPD21',
'SPD22',
'SPD23',
'SPD24',
'LVC00',
'LVC01',
'LVC02',
'LVC03',
'LVC04',
'LVC05',
'LVC06',
'LVC07',
'LVC08',
'LVC09',
'LVC10',
'LVC11',
'LVC12',
'LVC13',
'LVC14',
'LVC15',
'LVC16',
'LVC17',
'LVC18',
'LVC19',
'LVC20',
'LVC21',
'LVC22',
'LVC23',
'LVC24',
'AGROUP',
'FGROUP',
'RGROUP',
'NOTES',
'CAR00',
'CAR01',
'CAR02',
'CAR03',
'CAR04',
'CAR05',
'CAR06',
'CAR07',
'CAR08',
'CAR09',
'CAR10',
'CAR11',
'CAR12',
'CAR13',
'CAR14',
'CAR15',
'CAR16',
'CAR17',
'CAR18',
'CAR19',
'CAR20',
'CAR21',
'CAR22',
'CAR23'], inplace = True)

In [87]:
aadt_E = aadt[aadt['SRI_CMS']== '00000003__']
aadt_W = aadt[aadt['SRI_CMS']== '00000003_W']

In [88]:
ndf_E = ndf[ndf['direction']=='E']
ndf_W = ndf[ndf['direction']=='W']

In [89]:
aadt_E.index = pd.IntervalIndex.from_arrays(aadt_E['BEGINMP'], aadt_E['ENDMP'],closed='both')
aadt_W.index = pd.IntervalIndex.from_arrays(aadt_W['ENDMP'], aadt_W['BEGINMP'],closed='both')

In [90]:
ndf_E['begin_MP'] = ndf_E['MP'].apply(lambda x: aadt_E.iloc[aadt_E.index.get_loc(x)]['BEGINMP'])
ndf_W['begin_MP'] = ndf_W['MP'].apply(lambda x: aadt_W.iloc[aadt_W.index.get_loc(x)]['BEGINMP'])

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [91]:
aadt['DIR'] = aadt['SRI_CMS'].apply(lambda d: 'E' if d == '00000003__' else 'W')

In [92]:
aadt['BEGINMP'] = aadt['BEGINMP'].astype(str)

In [93]:
ndf_E['begin_MP'] = ndf_E['begin_MP'].astype(str)
ndf_W['begin_MP'] = ndf_W['begin_MP'].astype(str)

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [94]:
ndf1 = pd.merge(ndf_E, aadt, how = 'left', left_on = ['direction', 'begin_MP'], right_on = ['DIR', 'BEGINMP'])
ndf2 = pd.merge(ndf_W, aadt, how = 'left', left_on = ['direction', 'begin_MP'], right_on = ['DIR', 'BEGINMP'])

In [95]:
ndf = pd.concat([ndf1, ndf2]).reset_index(drop=True)

In [96]:
ndf['HOUR'] = ndf['timestamp'].dt.hour
ndf['HOUR'] = ndf['HOUR'].map("{:02}".format)

In [97]:
ndf['vol_AADT_ref'] = 'VOL' + ndf['HOUR']
ndf['trk_AADT_ref'] = 'TRK' + ndf['HOUR']

In [98]:
ndf['VOL_AADT'] = ndf.lookup(ndf.index, ndf['vol_AADT_ref']) 
ndf['TRK_AADT'] = ndf.lookup(ndf.index, ndf['trk_AADT_ref']) 

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  """Entry point for launching an IPython kernel.
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  


In [100]:
ndf['VOL'] = ndf['VOL_AADT']
ndf['VOL'] =np.where(ndf['weekday']==5, ndf['VOL_AADT']/ndf['WEEKEND_FACTOR'], ndf['VOL'])
ndf['VOL'] =np.where(ndf['weekday']==6, ndf['VOL_AADT']/ndf['WEEKEND_FACTOR'], ndf['VOL'])

In [101]:
ndf['TRK'] = ndf['TRK_AADT']
ndf['TRK'] =np.where(ndf['weekday']==5, ndf['TRK_AADT']/ndf['WEEKEND_FACTOR'], ndf['TRK'])
ndf['TRK'] =np.where(ndf['weekday']==6, ndf['TRK_AADT']/ndf['WEEKEND_FACTOR'], ndf['TRK'])

In [102]:
ndf['VOL'] = ndf['VOL'].round()
ndf['TRK'] = ndf['TRK'].round()

In [103]:
ndf.drop(columns = ['begin_MP',
'SRI_CMS',
'BEGINMP',
'ENDMP',
'DIST',
'VOL00',
'VOL01',
'VOL02',
'VOL03',
'VOL04',
'VOL05',
'VOL06',
'VOL07',
'VOL08',
'VOL09',
'VOL10',
'VOL11',
'VOL12',
'VOL13',
'VOL14',
'VOL15',
'VOL16',
'VOL17',
'VOL18',
'VOL19',
'VOL20',
'VOL21',
'VOL22',
'VOL23',
'VOL24',
'TRK00',
'TRK01',
'TRK02',
'TRK03',
'TRK04',
'TRK05',
'TRK06',
'TRK07',
'TRK08',
'TRK09',
'TRK10',
'TRK11',
'TRK12',
'TRK13',
'TRK14',
'TRK15',
'TRK16',
'TRK17',
'TRK18',
'TRK19',
'TRK20',
'TRK21',
'TRK22',
'TRK23',
'TRK24',
'WEEKEND_FACTOR',
'DIR',
'HOUR',
'vol_AADT_ref',
'trk_AADT_ref',
'VOL_AADT',
'TRK_AADT'
], inplace = True)

In [104]:
ndf.to_csv('./data/ndf_processed.csv')